In [1]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= '0.20'

from sklearn.impute import SimpleImputer
#from sklearn.pipeline import Pipelines
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay

# TensorFlow ≥2.0 is required
import tensorflow_addons as tfa
import tensorflow as tf
assert tf.__version__ >= '2.0'

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import plot_model

from keras.models import Sequential
#from keras.layers.wrappers import TimeDistributed
#from keras.layers import Dense,LSTM,Conv2D, BatchNormalization,Flatten, MaxPooling2D
#from keras.layers import Conv2DTranspose,Concatenate,UpSampling2D,Cropping2D
#from keras.layers import Input, Lambda, Reshape, Dropout, Activation

from tensorflow.keras.layers import Dropout, BatchNormalization, Reshape
from tensorflow.keras.layers import Dense, Conv2D, Input, MaxPooling2D, Flatten, MaxPool2D, MaxPool3D, UpSampling2D
from tensorflow.keras.layers import Conv2DTranspose, Flatten, Reshape, Cropping2D, Embedding, BatchNormalization,ZeroPadding2D
from tensorflow.keras.layers import LeakyReLU, Activation, Input, add, multiply
from tensorflow.keras.layers import concatenate, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers import Lambda
import tensorflow.keras.backend as K
from tensorflow.keras.models import load_model 
from tensorflow.python.ops import gen_nn_ops

#print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

# Common imports
import os
import glob
import numpy as np
import pandas as pd
import geopandas as gpd
import xarray as xr
import dask
import datetime
import math
import pathlib
import hashlib
dask.config.set({'array.slicing.split_large_chunks': False})

# To make this notebook's output stable across runs
np.random.seed(42)

# Config matplotlib
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt


mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)
# Dotenv
from dotenv import dotenv_values
# Custom utils
from utils.utils_data import *
from utils.utils_ml import *
from utils.utils_plot import *
from utils.utils_unet import *
from utils.utils_resnet import *

In [2]:
#load test data
dg_test_X = np.array(xr.open_dataarray('tmp/data/dg_test_X.nc'))
dg_test_Y = np.array(xr.open_dataarray('tmp/data/dg_test_Y.nc'))
dg_test_Y_xtrm = np.array(xr.open_dataarray('tmp/data/dg_test_Y_xtrm.nc'))

In [3]:
# Define args for the U-net model
i_shape = dg_test_X.shape[1:]
o_shape = (46,56)

print(f'X shape: {i_shape}')
print(f'y shape: {o_shape}')
output_channels = 1
num_filters = 32
use_batchnorm = True
dropout = True
lr = 0.0004
#optimizer = tf.optimizers.Adam(learning_rate = lr)
EPOCHS = 50

X shape: (46, 56, 10)
y shape: (46, 56)


In [4]:
names_mod = ['unet']
for imod in names_mod:
    tmp_file = pathlib.Path(f'tmp/{imod}')

In [5]:
custom_objects = {"weighted_cross_entropy_fn": weighted_binary_cross_entropy}
with keras.utils.custom_object_scope(custom_objects):
                m = load_model(tmp_file)

2022-04-19 19:43:41.853095: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-04-19 19:43:42.568198: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10415 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:0d:00.0, compute capability: 6.1


In [6]:
def rename(model, layer, new_name):
    def _get_node_suffix(name):
        for old_name in old_nodes:
            if old_name.startswith(name):
                return old_name[len(name):]

    old_name = layer.name
    old_nodes = list(model._network_nodes)
    new_nodes = []

    for l in model.layers:
        if l.name == old_name:
            l._name = new_name
            # vars(l).__setitem__('_name', new)  # bypasses .__setattr__
            new_nodes.append(new_name + _get_node_suffix(old_name))
        else:
            new_nodes.append(l.name + _get_node_suffix(l.name))
    model._network_nodes = set(new_nodes)

In [7]:
#rename(m, m.layers[len(m.layers)-1], 'predictions')

In [6]:
image =  dg_test_X[0,:,:,:]

In [7]:
alpha=2
epsilon=1e-7

In [27]:
from tensorflow.python.framework.ops import disable_eager_execution 
disable_eager_execution()

In [20]:
#import tensorflow.compat.v1 as tf
#tf.disable_v2_behavior()

In [10]:
class RelevancePropagation(object):
    """Very basic implementation of the layer-wise relevance propagation algorithm.
    """

    def __init__(self, model, epsilon, input_size, output_size):
        self.epsilon = epsilon
        # Load model
        input_shape = input_size
        self.model = model 
        self.rule = "z_plus"
        self.pooling_type = "max"

        # Extract model's weights
        self.weights = {weight.name.split('/')[0]: weight for weight in self.model.trainable_weights
                        if 'bias' not in weight.name}

        # Extract activation layers
        self.activations = [layer.output for layer in self.model.layers]
        self.activations = self.activations[::-1]

        # Extract the model's layers name
        self.layer_names = [layer.name for layer in self.model.layers]
        self.layer_names = self.layer_names[::-1]

        # Build relevance graph
        self.relevance = self.relevance_propagation()

    def run(self, image):
        """Computes feature relevance maps for a single image.

        Args:
            image: array of shape (W, H, C)

        Returns:
            RGB or grayscale relevance map.

        """
        f = K.function(inputs=self.model.input, outputs=self.relevance)
        image = preprocess_input(image)
        image = tf.expand_dims(image, axis=0)
        relevance_scores = f(inputs=image)
        relevance_scores = self.postprocess(relevance_scores)
        return np.squeeze(relevance_scores)

    def relevance_propagation(self):
        """Builds graph for relevance propagation."""
        relevance = self.model.output
        print(self.layer_names)
        for i, layer_name in enumerate(self.layer_names):
            print(i)
            print(layer_name)
            if 'predictions' in layer_name:
                #relevance = self.relprop_dense(self.activations[i+1], self.weights[layer_name], relevance)
                relevance = relevance
            elif 'fc' in layer_name:
                relevance = self.relprop_dense(self.activations[i+1], self.weights[layer_name], relevance)
            elif 'flatten' in layer_name:
                relevance = self.relprop_flatten(self.activations[i+1], relevance)
            elif 'pool' in layer_name:
                relevance = self.relprop_pool(self.activations[i+1], relevance)
            elif 'conv' in layer_name:
                relevance = self.relprop_conv(self.activations[i+1], self.weights[layer_name], relevance, layer_name)
            elif 'dropout' in layer_name or 'normalization' in layer_name or 'cropping' in layer_name or 'concatenate' in layer_name or 'activation' in layer_name or 'conv2d_transpose' in layer_name or 'zero_padding2d' in layer_name  :
            #elif 'input' in layer_name:
                pass
            else:
                raise Exception("Error: layer type not recognized.")
        return relevance

    def relprop_dense(self, x, w, r):
        """Implements relevance propagation rules for dense layers.

        Args:
            x: array of activations
            w: array of weights
            r: array of relevance scores

        Returns:
            array of relevance scores of same dimension as a

        """
        if self.rule == "z_plus":
            w_pos = tf.maximum(w, 0.0)
            z = tf.matmul(x, w_pos) + self.epsilon
            s = r / z
            c = tf.matmul(s, tf.transpose(w_pos))
            return c * x
        else:
            raise Exception("Error: rule for dense layer not implemented.")

    def relprop_flatten(self, x, r):
        """Transfers relevance scores coming from dense layers to last feature maps of network.

        Args:
            x: array of activations
            r: array of relevance scores

        Returns:
            array of relevance scores of same dimension as a

        """
        return tf.reshape(r, tf.shape(x))

    def relprop_pool(self, x, r, ksize=(1, 2, 2, 1), strides=(1, 2, 2, 1), padding='SAME'):
        """Implements relevance propagation through pooling layers.

        Args:
            x: array of activations
            r: array of relevance scores
            ksize: pooling kernel dimensions used during forward path
            strides: step size of pooling kernel used during forward path
            padding: parameter for SAME or VALID padding

        Returns:
            array of relevance scores of same dimensions as a

        """
        if self.pooling_type == "avg":
            z = tf.nn.avg_pool(x, ksize, strides, padding) + self.epsilon
            s = r / z
            c = gen_nn_ops.avg_pool_grad(tf.shape(x), s, ksize, strides, padding)
        elif self.pooling_type == "max":
            z = tf.nn.max_pool(x, ksize, strides, padding) + self.epsilon
            s = r / z
            c = gen_nn_ops.max_pool_grad_v2(x, z, s, ksize, strides, padding)
        else:
            raise Exception("Error: no such unpooling operation implemented.")
        return c * x

    def relprop_conv(self, x, w, r, name, strides=(1, 1, 1, 1), padding='SAME'):
        """Implements relevance propagation rules for convolutional layers.

        Args:
            x: array of activations
            w: array of weights
            r: array of relevance scores
            name: current layer name
            strides: step size of filters used during forward path
            padding: parameter for SAME or VALID padding

        Returns:
            array of relevance scores of same dimensions as a

        """
        if name == 'block1_conv1':
            x = tf.ones_like(x)     # only for input

        if self.rule == "z_plus":
            w_pos = tf.maximum(w, 0.0)
            z = tf.nn.conv2d(x, w_pos, strides, padding) + self.epsilon
            
            if (r.shape[1] > z.shape[1]):
                r_crop = crop_output(z, r)
                s = r_crop / z
            else:
                z_crop = crop_output(r, z)
                s = r / z_crop

            #s = r / z
            c = tf.compat.v1.nn.conv2d_backprop_input(tf.shape(x), w_pos, s, strides, padding)
            return c * x
        else:
            raise Exception("Error: rule for convolutional layer not implemented.")

    @staticmethod
    def rescale(x):
        """Rescales relevance scores of a batch of relevance maps between 0 and 1

        Args:
            x: RGB or grayscale relevance maps with dimensions (N, W, H, C) or (N, W, H), respectively.

        Returns:
            Rescaled relevance maps of same dimensions as input

        """
        x_min = np.min(x, axis=(1, 2), keepdims=True)
        x_max = np.max(x, axis=(1, 2), keepdims=True)
        return (x - x_min).astype("float64") / (x_max - x_min).astype("float64")

    def postprocess(self, x):
        """Postprocesses batch of feature relevance scores (relevance_maps).

        Args:
            x: array with dimension (N, W, H, C)

        Returns:
            x: array with dimensions (N, W, H, C) or (N, W, H) depending on if grayscale or not

        """
        if self.grayscale:
            x = np.mean(x, axis=-1)
        x = self.rescale(x)
        return x


In [11]:
lrp = RelevancePropagation(m, epsilon=epsilon, input_size=i_shape, output_size=o_shape)

['cropping2d_9', 'conv2d_37', 'activation_31', 'conv2d_36', 'dropout_9', 'activation_30', 'conv2d_35', 'concatenate_7', 'cropping2d_8', 'conv2d_transpose_7', 'activation_29', 'conv2d_34', 'dropout_8', 'activation_28', 'conv2d_33', 'concatenate_6', 'cropping2d_7', 'conv2d_transpose_6', 'activation_27', 'conv2d_32', 'dropout_7', 'activation_26', 'conv2d_31', 'concatenate_5', 'cropping2d_6', 'conv2d_transpose_5', 'activation_25', 'conv2d_30', 'dropout_6', 'activation_24', 'conv2d_29', 'concatenate_4', 'cropping2d_5', 'conv2d_transpose_4', 'batch_normalization_3', 'conv2d_28', 'dropout_5', 'batch_normalization_2', 'conv2d_27', 'max_pooling2d_7', 'activation_23', 'conv2d_26', 'spatial_dropout2d_7', 'activation_22', 'conv2d_25', 'max_pooling2d_6', 'activation_21', 'conv2d_24', 'spatial_dropout2d_6', 'activation_20', 'conv2d_23', 'max_pooling2d_5', 'activation_19', 'conv2d_22', 'spatial_dropout2d_5', 'activation_18', 'conv2d_21', 'max_pooling2d_4', 'activation_17', 'conv2d_20', 'spatial_dropo

TypeError: You are passing KerasTensor(type_spec=TensorSpec(shape=(None, 6, 8, 512), dtype=tf.float32, name=None), name='activation_23/Relu:0', description="created by layer 'activation_23'"), an intermediate Keras symbolic input/output, to a TF API that does not allow registering custom dispatchers, such as `tf.cond`, `tf.function`, gradient tapes, or `tf.map_fn`. Keras Functional model construction only supports TF API calls that *do* support dispatching, such as `tf.math.add` or `tf.reshape`. Other APIs cannot be called directly on symbolic Kerasinputs/outputs. You can work around this limitation by putting the operation in a custom Keras layer `call` and calling that layer on this symbolic input/output.

In [59]:
class LayerwiseRelevancePropagation:

    """Very basic implementation of the layer-wise relevance propagation algorithm.
    """

    def __init__(self, model, epsilon, input_size, output_size):
        self.epsilon = epsilon
        # Load model
        input_shape = input_size
        self.model = model 
        self.pooling_type = "max"
        self.alpha = alpha
        self.beta = 1 - alpha
        self.epsilon = epsilon

        self.names, self.activations, self.weights = get_model_params(self.model)
        self.num_layers = len(self.names)

        self.relevance = self.compute_relevances()
        #self.lrp_runner = K.function(inputs=[self.model.input, ], outputs=[self.relevance, ])
        
        
    def compute_relevances(self):
        r = self.model.output
        print(self.num_layers)
        for i in range(self.num_layers - 2, -1, -1):
            print(i)
            # note: max_pooling is not working, need to figure this out
            if 'dense' in self.names[i + 1]:
                r = backprop_fc(self.weights[i + 1][0], self.weights[i + 1][1], self.activations[i], r)
            elif 'flatten' in self.names[i + 1]:
                r = backprop_flatten(self.activations[i], r)
            elif 'pool' in self.names[i + 1]:
                r = backprop_max_pool2d(self.activations[i], r)
            elif 'conv2d' in self.names[i + 1]:
                r = backprop_conv2d(self.weights[i + 1][0], self.weights[i + 1][1], self.activations[i], r)
            #elif 'conv2d_transpose' in names[i + 1]:
            #    r = backprop_conv2d(weights[i + 1][0], weights[i + 1][1], activations[i], r, strides=strides[i+1])
            elif 'conv1d' in self.names[i + 1]:
                r = backprop_conv1d(self.weights[i + 1][0], self.weights[i + 1][1], self.activations[i], r, strides=strides[i+1])
            elif 'dropout' in self.names[i + 1] or 'normalization' in self.names[i + 1] or 'cropping' in self.names[i + 1] or 'concatenate' in self.names[i + 1] or 'activation' in self.names[i + 1] or 'conv2d_transpose' in self.names[i + 1] or 'zero_padding2d' in self.names[i + 1]  :
            #elif 'input' in layer_name:
                pass
            else:
                raise Exception("Error: layer type not recognized.")
           
            return r

    def backprop_fc(self, w, b, a, r):
        w_p = K.maximum(w, 0.)
        b_p = K.maximum(b, 0.)
        z_p = K.dot(a, w_p) + b_p + self.epsilon
        s_p = r / z_p
        c_p = K.dot(s_p, K.transpose(w_p))

        w_n = K.minimum(w, 0.)
        b_n = K.minimum(b, 0.)
        z_n = K.dot(a, w_n) + b_n - self.epsilon
        s_n = r / z_n
        c_n = K.dot(s_n, K.transpose(w_n))

        return a * (self.alpha * c_p + self.beta * c_n)

    def backprop_flatten(self, a, r):
        shape = a.get_shape().as_list()
        shape[0] = -1
        return K.reshape(r, shape)

    def backprop_max_pool2d(self, a, r, ksize=(1, 2, 2, 1), strides=(1, 2, 2, 1)):
        z = K.pool2d(a, pool_size=ksize[1:-1], strides=strides[1:-1], padding='valid', pool_mode='max')

        z_p = K.maximum(z, 0.) + self.epsilon
        s_p = r / z_p
        c_p = gen_nn_ops.max_pool_grad_v2(a, z_p, s_p, ksize, strides, padding='VALID')

        z_n = K.minimum(z, 0.) - self.epsilon
        s_n = r / z_n
        c_n = gen_nn_ops.max_pool_grad_v2(a, z_n, s_n, ksize, strides, padding='VALID')

        return a * (self.alpha * c_p + self.beta * c_n)

    def backprop_conv2d(self, w, b, a, r, strides=(1, 1, 1, 1)):
        w_p = K.maximum(w, 0.)
        b_p = K.maximum(b, 0.)
        z_p = K.conv2d(a, kernel=w_p, strides=strides[1:-1], padding='same') + b_p + self.epsilon
        if (r.shape[1] > z_p.shape[1]):
            r_p_crop = crop_output(z_p, r)
            s_p = r_p_crop / z_p
        else:
            z_p_crop = crop_output(r, z_p)
            s_p = r / z_p_crop
        #s_p = r / z_p
        c_p = K.tf.nn.conv2d_backprop_input(K.shape(a), w_p, s_p, strides, padding='SAME')

        #w_n = K.minimum(w, 0.)
        #b_n = K.minimum(b, 0.)
        #z_n = K.conv2d(a, kernel=w_n, strides=strides[1:-1], padding='same') + b_n - self.epsilon
        #s_n = r / z_n
        #c_n = K.tf.nn.conv2d_backprop_input(K.shape(a), w_n, s_n, strides, padding='SAME')

        #return a * (self.alpha * c_p + self.beta * c_n)
        return  a * c_p

    def predict_labels(self, images):
        return predict_labels(self.model, images)

    def run_lrp(self, images):
        print("Running LRP on {0} images...".format(len(images)))
        return self.lrp_runner([images, ])[0]


In [60]:
#lrp = RelevancePropagation(model, epsilon=epsilon, input_size=i_shape, output_size=o_shape)

In [61]:
def get_model_params(model):
    names, activations, weights = [], [], []
    for layer in model.layers:
        name = layer.name if layer.name != 'predictions' else 'fc_out'
        names.append(name)
        activations.append(layer.output)
        weights.append(layer.get_weights())
    return names, activations, weights

In [63]:
lrp=LayerwiseRelevancePropagation(m, epsilon=epsilon, input_size=i_shape, output_size=o_shape)

65
63


In [ ]:
for i in range( - 2, -1, -1):
            print(i)